<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MEDICAL_ADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-aiplatform[adk,agent_engines] -q
!pip install colab-env -q

In [2]:
# Import libraries from the Agent Development Kit
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()

In [ ]:
from google.cloud import aiplatform
import colab_env
import os

# Get project details from environment variables
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
REGION = os.environ.get("GOOGLE_CLOUD_REGION")
SERVICEACCOUNT = os.environ.get("GOOGLE_CLOUD_SERVICEACCOUNT")
PROJECT_NUMBER = os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
STAGING_BUCKET = f"gs://{BUCKET_NAME}/staging"


GOOGLE_GENAI_USE_VERTEXAI=1
GOOGLE_CLOUD_PROJECT=PROJECT_ID
GOOGLE_CLOUD_LOCATION=REGION

In [20]:
#MODEL='gemini-2.0-flash-001'
MODEL='gemini-2.5-pro-preview-05-06'
# Create a basic agent with name, description, and instructions only
basic_agent = Agent(
    model=MODEL,
    name="medical_agent",
    description="This agent responds to medical inquiries.",
    #instruction="Respond to user questions accurately and concisely based on the information provided in the query.", # Changed instruction
    instruction = "Identify and extract the specific term that the query is defining. Provide only that term as your answer.",
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [21]:
AGENT_APP_NAME = 'medical_agent'

def send_query_to_agent(agent, query):
  session = session_service.create_session(app_name=AGENT_APP_NAME,
                                           user_id='user',)

  content = types.Content(role='user', parts=[types.Part(text=query)])
  runner = Runner(app_name=AGENT_APP_NAME,
                agent=agent,
                artifact_service=artifact_service,
                session_service=session_service)
  events = runner.run(user_id='user',
                    session_id=session.id,
                    new_message=content,)

  for _, event in enumerate(events):
    is_final_response = event.is_final_response()
    if is_final_response:
        final_response = event.content.parts[0].text
        print(f"Final Response: {final_response}")

In [24]:
query0="while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in summary dor is found to be a clinical challenge in the practice of fertility care with controversial countermeasures to prevent or treat the condition nevertheless some promising measure such as oocyte embryo and tissue cryopreservation ovarian transplantation dietary supplementation and the transfer of mitochondria have offered hopes towards ameliorating the burden of dor this review attempts to discuss dor from different perspectives and summarize some existing hopes in clinical practice"

query1 = "In the context of ovarian reserve, what is the full term represented by the abbreviation AFC?"


In [26]:
print(f"Query:\n{query0}")
send_query_to_agent(basic_agent,  query0)
print('\n\n')

print(f"Query:\n{query1}")
send_query_to_agent(basic_agent,  query1)
print('\n\n')

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 